## Day 8

https://adventofcode.com/2021/day/8


In [1]:
def readInput8(filename):
    with open(filename) as f:
        return [ [ p.split() for p in l.strip('\n').split(" | ") ] for l in f.readlines() ]

i0 = readInput8("data/day08test01.txt")
i1 = readInput8("data/day08test02.txt")
i2 = readInput8("data/input08.txt")

### Part 1
Because the digits 1, 4, 7, and 8 each use a unique number of segments, you should be able to tell which combinations of signals correspond to those digits. Count only digits in the output values.

In [2]:
from collections import Counter

def countUnique(o):
    # 1 = 2S ; 4 = 4S ; 7 = 3S ; 8 = 7S
    c = Counter([ len(d) for d in o ])
    return c[2]+c[4]+c[3]+c[7]

def part1(i0):
    return sum([ countUnique(o) for i,o in i0 ])

print("Test 1:",part1(i1))
print("Part 1:",part1(i2))

Test 1: 26
Part 1: 512


### Part 2 brute force

Not trying to be smart, probing all wiring permutations for each line...

In [7]:
from itertools import permutations

display = {}
display["abcefg"]  = 0
display["cf"]      = 1
display["acdeg"]   = 2
display["acdfg"]   = 3
display["bcdf"]    = 4
display["abdfg"]   = 5
display["abdefg"]  = 6
display["acf"]     = 7
display["abcdefg"] = 8
display["abcdfg"]  = 9

def checkWiring(di):
    return sum([1 if i in display.keys() else 0 for i in di ])==len(di)

def part2BruteForce(i0,verbose=True):
    output = 0
    origin = 'abcdefg'
    do = ""
    for i,o in i0:
        # find correct wiring for current line
        for p in permutations(origin):
            scramb = "".join(p)
            # generate wiring for given permutation
            wiring = {}
            for ori,scr in zip(origin,scramb):
                wiring[scr] = ori
            # test wiring on line
            di = [ "".join(sorted([ wiring[s] for s in ii ])) for ii in i ]
            do = [ "".join(sorted([ wiring[s] for s in oo ])) for oo in o ]
            dd = list(dict.fromkeys(di+do)) # remove duplicates
            if checkWiring(dd): break
        # output value
        value = int("".join(str(display[oo]) for oo in do))
        if verbose: 
            for _o in o:
                print("{:7s} ".format(_o),end=" ")
            print(value)
        output += value
    return output

In [8]:
part2BruteForce(i0)

cdfeb    fcadb    cdfeb    cdbaf    5353


5353

In [9]:
part2BruteForce(i1)

fdgacbe  cefdb    cefbgd   gcbe     8394
fcgedb   cgb      dgebacf  gc       9781
cg       cg       fdcagb   cbg      1197
efabcd   cedba    gadfec   cb       9361
gecf     egdcabf  bgf      bfgea    4873
gebdcfa  ecba     ca       fadegcb  8418
cefg     dcbef    fcge     gbcadfe  4548
ed       bcgafe   cdgba    cbgef    1625
gbdfcae  bgc      cg       cgb      8717
fgae     cfgab    fg       bagce    4315


61229

In [10]:
part2BruteForce(i2,False)

1091165

### Part 2 clever search

I can actually use the univoque number of segments associated to certain numbers (1, 4, 7) to dramatically reduce the search space!

In [14]:
def checkWiring(di):
    return sum([1 if i in display.keys() else 0 for i in di ])==len(di)

def part2SearchFast(i0,verbose=True):
    output = 0
    do = ""
    for i,o in i0:
        foundWiring = False
        # remove duplicates
        io = list(dict.fromkeys(i+o))
        # find unique segments
        io_len = [ len(s) for s in io]
        s1 = io[io_len.index(2)]
        s7 = "".join([c for c in io[io_len.index(3)] if c not in s1 ]) # segment of 7 not in common with 1's
        s4 = "".join([c for c in io[io_len.index(4)] if c not in s1 ]) # segment of 4 not in common with 1's
        for p1 in permutations(s1): # permutations for 1 segments (to be associated to 'cf')
            w1 = {}
            for ori,scr in zip('cf',p1):
                w1[scr] = ori
            # add permutations of unique 4 segments (to be associated to 'bd')
            for p4 in permutations(s4): 
                w4 = w1.copy()
                for ori,scr in zip('bd',p4):
                    w4[scr] = ori
                # add unique 7 segment (to be associated to 'a')
                w4[s7] = 'a'      
                # add permutations of remaining segments (to be associated to 'eg')
                slast = ''.join([ c for c in 'abcdefg' if c not in w4.keys() ])                
                for p in permutations(slast):
                    wiring = w4.copy()
                    for ori,scr in zip('eg',p):
                        wiring[scr] = ori
                    # test wiring on line
                    di = [ "".join(sorted([ wiring[s] for s in ii ])) for ii in i ]
                    do = [ "".join(sorted([ wiring[s] for s in oo ])) for oo in o ]
                    dd = list(dict.fromkeys(di+do)) # remove duplicates
                    if checkWiring(dd):
                        foundWiring = True
                        break
                if foundWiring: break
            if foundWiring: break      
        if foundWiring:
            # output value
            value = int("".join(str(display[oo]) for oo in do))
            if verbose: 
                for _o in o:
                    print("{:7s} ".format(_o),end=" ")
                print(value)
            output += value
        else:
            return -1
    return output

In [15]:
part2SearchFast(i0)

cdfeb    fcadb    cdfeb    cdbaf    5353


5353

In [16]:
part2SearchFast(i1)

fdgacbe  cefdb    cefbgd   gcbe     8394
fcgedb   cgb      dgebacf  gc       9781
cg       cg       fdcagb   cbg      1197
efabcd   cedba    gadfec   cb       9361
gecf     egdcabf  bgf      bfgea    4873
gebdcfa  ecba     ca       fadegcb  8418
cefg     dcbef    fcge     gbcadfe  4548
ed       bcgafe   cdgba    cbgef    1625
gbdfcae  bgc      cg       cgb      8717
fgae     cfgab    fg       bagce    4315


61229

In [17]:
part2SearchFast(i2,False)

1091165

### Part 2 procedural 1: finding the wiring

The problem can actually be solved with no search but in a procedural way, as a Sudoku puzzle!

In [18]:
def findWiring(i,o):
    wiring = {}
    io = sorted(list(dict.fromkeys([ "".join(sorted(s)) for s in i+o ])),key=len)
    io_len = [ len(s) for s in io ]
    s8 = "abcdefg"
    s1 = io[io_len.index(2)] # cf
    s7 = io[io_len.index(3)] # acf
    s4 = io[io_len.index(4)] # bcdf
    s235 = [ s for s in io if len(s)==5 ]
    s069 = [ s for s in io if len(s)==6 ]
    a = "".join([ s for s in s7 if s not in s1 ])
    wiring[a] = 'a'
    bd = "".join([ s for s in s4 if s not in s1 ])
    s5 = "" # abdfg
    for n in s235:
        if sum([n.count(m) for m in bd])==2:
            s5 = n
            break
    f = "".join([ s for s in s5 if s in s1 ]) 
    wiring[f] = 'f'
    c = "".join([ s for s in s1 if s != f ]) 
    wiring[c] = 'c'
    ag = "".join([ s for s in s5 if s not in s4 ])
    g = "".join([ s for s in ag if s != a ])
    wiring[g] = 'g'
    e = "".join([ s for s in s8 if s not in s4 and s!=a and s!=g])
    wiring[e] = 'e'
    cde = "".join([ s for s in s069[0] if s not in s069[1] ]+[ s for s in s069[1] if s not in s069[2] ]+[ s for s in s069[2] if s not in s069[0] ])
    d = "".join([ s for s in cde if s!=c and s!=e ])
    wiring[d] = 'd'
    b = "".join([ s for s in s8 if s not in wiring.keys() ])
    wiring[b] = 'b'
    return wiring

def part2ProceduralWiring(i0,verbose=True):
    output = 0
    for i,o in i0:
        wiring = findWiring(i,o)
        do = [ "".join(sorted([ wiring[s] for s in oo ])) for oo in o ]
        value = int("".join(str(display[oo]) for oo in do))
        if verbose:
            for _o in o:
                print("{:7s} ".format(_o),end=" ")
            print(value)
        output += value
    return output

In [19]:
part2ProceduralWiring(i0)

cdfeb    fcadb    cdfeb    cdbaf    5353


5353

In [20]:
part2ProceduralWiring(i1)

fdgacbe  cefdb    cefbgd   gcbe     8394
fcgedb   cgb      dgebacf  gc       9781
cg       cg       fdcagb   cbg      1197
efabcd   cedba    gadfec   cb       9361
gecf     egdcabf  bgf      bfgea    4873
gebdcfa  ecba     ca       fadegcb  8418
cefg     dcbef    fcge     gbcadfe  4548
ed       bcgafe   cdgba    cbgef    1625
gbdfcae  bgc      cg       cgb      8717
fgae     cfgab    fg       bagce    4315


61229

In [21]:
part2ProceduralWiring(i2,False)

1091165

### Part 2 procedural 2: directly finding the number mapping

In [80]:
def part2ProceduralNumbers(i0,verbose=True):
    output = 0
    
    for i,o in i0:
        io = sorted(list(dict.fromkeys([ "".join(sorted(s)) for s in i+o ])),key=len)
        io_len = [ len(s) for s in io ]
        s1 = set(io[io_len.index(2)])
        s7 = set(io[io_len.index(3)])
        s4 = set(io[io_len.index(4)])
        s8 = set(io[io_len.index(7)])
        s069 = [ set(s) for s in io if len(s)==6 ]
        s9 = [ s for s in s069 if s4<s ][0]
        s0 = [ s for s in s069 if s1<s and s!=s9 ][0]
        s6 = [ s for s in s069 if s!=s0 and s!=s9][0]
        s235 = [ set(s) for s in io if len(s)==5 ]
        s3 = [ s for s in s235 if s1<s ][0]
        s5 = [ s for s in s235 if s<s6 ][0]
        s2 = [ s for s in s235 if s!=s3 and s!=s5 ][0]

        mapping = {}
        mapping["".join(sorted(s0))] = 0
        mapping["".join(sorted(s1))] = 1
        mapping["".join(sorted(s2))] = 2
        mapping["".join(sorted(s3))] = 3
        mapping["".join(sorted(s4))] = 4
        mapping["".join(sorted(s5))] = 5
        mapping["".join(sorted(s6))] = 6
        mapping["".join(sorted(s7))] = 7
        mapping["".join(sorted(s8))] = 8
        mapping["".join(sorted(s9))] = 9

        do = [ mapping[ "".join(sorted(oo)) ] for oo in o ]
        value = int("".join(str(oo) for oo in do))
        if verbose:
            for _o in o:
                print("{:7s} ".format(_o),end=" ")
            print(value)
        output += value
    
    return output

In [81]:
part2ProceduralNumbers(i0)

cdfeb    fcadb    cdfeb    cdbaf    5353


5353

In [82]:
part2ProceduralNumbers(i1)

fdgacbe  cefdb    cefbgd   gcbe     8394
fcgedb   cgb      dgebacf  gc       9781
cg       cg       fdcagb   cbg      1197
efabcd   cedba    gadfec   cb       9361
gecf     egdcabf  bgf      bfgea    4873
gebdcfa  ecba     ca       fadegcb  8418
cefg     dcbef    fcge     gbcadfe  4548
ed       bcgafe   cdgba    cbgef    1625
gbdfcae  bgc      cg       cgb      8717
fgae     cfgab    fg       bagce    4315


61229

In [84]:
part2ProceduralNumbers(i2,False)

1091165